In [ ]:
!pip install --upgrade statsmodels  dask[dataframe]

In [ ]:
import pandas as pd
import dask.dataframe as dd
from enum import unique
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statsmodels.api as sm

In [ ]:
# Connecting to googledrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ruta = '/content/drive/MyDrive/celes_test/data_challenge_preprocess.parquet'
df = dd.read_parquet(ruta,
                     parse_dates=["Date"],
                     engine = 'pyarrow'
                 )

# Sort by Date to maintain time series order
df = df.sort_values("Date").persist()

# Verify sorting
df.shape[0].compute()

4978868

In [ ]:
df.compute()

,Date,ProductID,StoreID,ProductName,Quantity,Price,Month,Year,Weekday,week_of_year,year_month
0,2022-08-01,2e86ec5da9dd,4b429145d0d6,Ebiten maki and Pepper Black Coarse,0.9234,8.033580,8,2022,0,31,2022-08
4501,2022-08-01,1f28832cd0d2,dd0ce0c8cfa2,Salmon nigiri and Allspice Ground,0.9234,2941.029000,8,2022,0,31,2022-08
4500,2022-08-01,463bb3d2c83a,14d166b132ac,Cheeseburger and Amchoor,44.3232,7.294860,8,2022,0,31,2022-08
4499,2022-08-01,ac82e2ce8e0a,dd0ce0c8cfa2,Vegetable Soup and Paprika Smoked,0.9234,5811.879600,8,2022,0,31,2022-08
4498,2022-08-01,88f2a8f8004d,7eb078ba1297,Fettuccine Alfredo and Lemon Grass,0.9234,6898.721400,8,2022,0,31,2022-08
...,...,...,...,...,...,...,...,...,...,...,...
4975358,2024-09-25,77b0b19616b7,702c4fdea66b,Pasta with Tomato and Basil and Curry Hot,0.9234,5.087934,9,2024,2,39,2024-09
4975357,2024-09-25,93c66d7ed8c6,3b645acc7bbe,Chicken milanese and Peppercorns White,0.9234,7.433370,9,2024,2,39,2024-09
4975356,2024-09-25,463bb3d2c83a,702c4fdea66b,Cheeseburger and Amchoor,22.1616,3.834418,9,2024,2,39,2024-09
4975354,2024-09-25,e6dc17d84357,8ecfffa75c21,Califlower penne and Pimento Ground,0.9234,6.020568,9,2024,2,39,2024-09


### Convert Date column to date format

In [ ]:
df['Date'] = dd.to_datetime(df['Date'], format = '%Y-%m-%d')

### monthly products

In [ ]:
monthly_products = df.groupby(['Year', 'Month', 'ProductID']).agg(
    {"Quantity": "sum"}).sort_values(
        by = ['Year', 'Month','Quantity'], ascending = False).reset_index()

In [ ]:
monthly_products.shape[0].compute()

4852

In [ ]:
monthly_products.compute()

,Year,Month,ProductID,Quantity
0,2024,9,463bb3d2c83a,184912.6968
1,2024,9,16a562fb5931,49220.9136
2,2024,9,f9e43b9a154e,30807.3942
3,2024,9,93c66d7ed8c6,19115.3034
4,2024,9,77b0b19616b7,18601.8930
...,...,...,...,...
4847,2022,8,ad2ebfee51bf,0.9234
4848,2022,8,adeba912ff56,0.9234
4849,2022,8,b4a68da8b9e5,0.9234
4850,2022,8,dfef298b305e,0.9234


In [ ]:
# Group by 'Year' and 'Month' and take the first 10 rows for each group
first_10_per_month = monthly_products.groupby(['Year', 'Month']).head(10).sort_values(
        by = ['Year', 'Month','Quantity'], ascending = False)

# If you need to materialize the result, compute it:
first_10_per_month.compute()

,Year,Month,ProductID,Quantity
0,2024,9,463bb3d2c83a,184912.6968
1,2024,9,16a562fb5931,49220.9136
2,2024,9,f9e43b9a154e,30807.3942
3,2024,9,93c66d7ed8c6,19115.3034
4,2024,9,77b0b19616b7,18601.8930
...,...,...,...,...
4662,2022,8,98a7469435bb,10387.3266
4663,2022,8,59c2de4b66d6,8673.4962
4664,2022,8,e6dc17d84357,8178.5538
4665,2022,8,0a15b6462cf0,7824.8916


In [ ]:
first_10_per_month = first_10_per_month.compute() # Materialize the result to avoid issues with delayed computation

In [ ]:
first_10_per_month['Top_10'] = 1

In [ ]:
first_10_per_month.head()

,Year,Month,ProductID,Quantity,Top_10
0,2024,9,463bb3d2c83a,184912.6968,1
1,2024,9,16a562fb5931,49220.9136,1
2,2024,9,f9e43b9a154e,30807.3942,1
3,2024,9,93c66d7ed8c6,19115.3034,1
4,2024,9,77b0b19616b7,18601.8930,1


In [ ]:
df_promo = first_10_per_month[['Year', 'Month', 'ProductID', 'Top_10']].copy()
df_promo['Year'] = df_promo['Year'] + 1
df_promo = df_promo.rename(columns={'Top_10': 'Promotion'})

In [ ]:
df_promo.head()

,Year,Month,ProductID,Promotion
0,2025,9,463bb3d2c83a,1
1,2025,9,16a562fb5931,1
2,2025,9,f9e43b9a154e,1
3,2025,9,93c66d7ed8c6,1
4,2025,9,77b0b19616b7,1


In [ ]:
# Merge to mark rows that should have the promotion.
monthly_products = monthly_products.merge(df_promo, on=['Year', 'Month', 'ProductID'], how='left')

/usr/local/lib/python3.11/dist-packages/dask/dataframe/multi.py:169: UserWarning: Merging dataframes with merge column data type mismatches: 
+--------------------+------------+-------------+
| Merge columns      | left dtype | right dtype |
+--------------------+------------+-------------+
| ('Year', 'Year')   | float64    | int32       |
| ('Month', 'Month') | float64    | int32       |
+--------------------+------------+-------------+
Cast dtypes explicitly to avoid unexpected results.
  warnings.warn(


In [ ]:
monthly_products.head()

,Year,Month,ProductID,Quantity,Promotion
0,2024,9,463bb3d2c83a,184912.6968,1.0
1,2024,9,16a562fb5931,49220.9136,1.0
2,2024,9,f9e43b9a154e,30807.3942,1.0
3,2024,9,93c66d7ed8c6,19115.3034,1.0
4,2024,9,77b0b19616b7,18601.8930,1.0


In [ ]:
monthly_products['Promotion'] = monthly_products['Promotion'].fillna(0)

In [ ]:
monthly_products.compute()

,Year,Month,ProductID,Quantity,Promotion
0,2024,9,463bb3d2c83a,184912.6968,1.0
1,2024,9,16a562fb5931,49220.9136,1.0
2,2024,9,f9e43b9a154e,30807.3942,1.0
3,2024,9,93c66d7ed8c6,19115.3034,1.0
4,2024,9,77b0b19616b7,18601.8930,1.0
...,...,...,...,...,...
4847,2022,8,ad2ebfee51bf,0.9234,0.0
4848,2022,8,adeba912ff56,0.9234,0.0
4849,2022,8,b4a68da8b9e5,0.9234,0.0
4850,2022,8,dfef298b305e,0.9234,0.0


### Generating promotions

In [ ]:
store_product_monthly = df.groupby(['StoreID', 'ProductID', 'Year', 'Month']).agg(
    {"Quantity": "sum", 'Price': 'median'}).reset_index()

In [ ]:
store_product_monthly.shape[0].compute()

179775

In [ ]:
store_product_monthly = store_product_monthly.merge(monthly_products[['Year', 'Month', 'ProductID', 'Promotion']],
              on=['ProductID', 'Year', 'Month'], how='left')

In [ ]:
store_product_monthly.shape[0].compute()

179775

In [ ]:
store_product_monthly['Promotion'].value_counts().compute()

,count
Promotion,
0.0,168866
1.0,10909


In [ ]:
store_product_monthly = store_product_monthly.assign(
    Price_Promo = store_product_monthly['Price'].where(store_product_monthly['Promotion']==0, store_product_monthly['Price'] * 0.85)
)

In [ ]:
store_product_monthly[(store_product_monthly['Promotion'] == 1) & (store_product_monthly['Year'] == 2023)].head()

,StoreID,ProductID,Year,Month,Quantity,Price,Promotion,Price_Promo
78506,90f0f4bb4d8f,463bb3d2c83a,2023,8,211.4586,5.475762,1.0,4.654398
78507,7b247f240c80,463bb3d2c83a,2023,8,123.7356,5.335713,1.0,4.535356
78509,d3d274df729b,463bb3d2c83a,2023,8,473.7042,8.033580,1.0,6.828543
78510,250e620ccc15,f9e43b9a154e,2023,8,2078.5734,1233.777825,1.0,1048.711151
78513,18451d418d52,463bb3d2c83a,2023,8,3437.8182,5.320836,1.0,4.522711


### Creating the variable Year-Month as a Date variable from the variables Year and Month

In [ ]:
# Creating the variable Year-Month from the variables Year and Month
store_product_monthly['Year-Month'] = dd.to_datetime(store_product_monthly[['Year', 'Month']].assign(day=1))

### Checking null values

In [ ]:
store_product_monthly.isnull().sum().compute()/store_product_monthly.shape[0].compute()

,0
StoreID,0.0
ProductID,0.0
Year,0.0
Month,0.0
Quantity,0.0
Price,0.0
Promotion,0.0
Price_Promo,0.0
Year-Month,0.0


### Selecting final variables to the model

In [ ]:
final_variables = ['Year-Month', 'ProductID', 'StoreID', 'Quantity', 'Promotion', 'Price','Price_Promo']
df_final = store_product_monthly[final_variables]
df_final.head()

,Year-Month,ProductID,StoreID,Quantity,Promotion,Price,Price_Promo
0,2022-08-01,2e86ec5da9dd,4b429145d0d6,1432.1934,0.0,8.21826,8.21826
1,2022-08-01,1f28832cd0d2,dd0ce0c8cfa2,2.7702,0.0,2646.46440,2646.46440
2,2022-08-01,463bb3d2c83a,14d166b132ac,7458.3018,0.0,7.29486,7.29486
3,2022-08-01,ac82e2ce8e0a,dd0ce0c8cfa2,117.2718,0.0,5811.87960,5811.87960
4,2022-08-01,88f2a8f8004d,7eb078ba1297,35.0892,0.0,6898.72140,6898.72140


In [ ]:
df_final.to_parquet('/content/drive/MyDrive/celes_test/data-challenge-final-selection.parquet')